<img src="../../../img/logo-bdc.png" align="right" width="64"/>

# <span style="color:#336699">Web Time Series Service (WTSS) - Examples</span>
<hr style="border:2px solid #0077b9;">

<div style="text-align: left;">
    <a href="https://nbviewer.jupyter.org/github/brazil-data-cube/code-gallery/blob/master/jupyter/Python/wtss/wtss-examples.ipynb"><img src="https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg" align="center"/></a>
</div>

<br/>

<div style="text-align: center;font-size: 90%;">
    Rennan F. B. Marujo<sup><a href="https://orcid.org/0000-0002-0082-9498"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Matheus Zaglia<sup><a href="https://orcid.org/0000-0001-6181-2158"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Gilberto R. Queiroz<sup><a href="https://orcid.org/0000-0001-7534-0219"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:brazildatacube@inpe.br">brazildatacube@inpe.br</a>
    <br/><br/>
    Last Update: March 12, 2021
</div>

<br/>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
<b>Abstract.</b> This Jupyter Notebook gives shows how to use the WTSS service to extract time series from <em>Brazil Data Cube</em>' service and how to perform a basic time series manipulation.
</div>    

<br/>
<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>This Jupyter Notebook is a supplement to the following paper:</b>
    <div style="margin-left: 10px; margin-right: 10px">
    Vinhas, L.; Queiroz, G. R.; Ferreira, K. R.; Camara, G. <a href="http://www.seer.ufu.br/index.php/revistabrasileiracartografia/article/view/44004" target="_blank">Web Services for Big Earth Observation Data</a>. Revista Brasileira de Cartografia, v. 69, n. 5, 18 maio 2017.
    </div>
</div>

# Python Client API
<hr style="border:1px solid #0077b9;">

If you haven't installed the [WTSS client for Python](https://github.com/brazil-data-cube/wtss.py), install it with `pip`:

In [ ]:
#!pip install wtss[matplotlib]

For more information on [WTSS client for Python](https://github.com/brazil-data-cube/wtss.py), see the introductory Jupyter Notebook about [Web Time Series Service (WTSS)](./wtss-introduction.ipynb) Introduction notebook.

# Set the service and Search for time series
<hr style="border:1px solid #0077b9;">

Import the WTSS client library:

In [ ]:
from wtss import WTSS

Define the service to be used:

In [ ]:
service = WTSS('https://brazildatacube.dpi.inpe.br/', access_token='change-me')

In [ ]:
service.coverages

Let's access the CBERS-4/AWFI coverage using the `CB4_64_16D_STK-1` product:

In [ ]:
cbers4_coverage = service['CB4_64_16D_STK-1']
cbers4_coverage

CBERS-4/AWFI spectral bands 15 and 16 correspond to the red and near-infrared (NIR) wavelength regions, respectivelly:

In [ ]:
red_band = 'BAND15'
nir_band = 'BAND16'

Let's retrieve the time series for data product `CB4_64_16D_STK-1`, in the location of `latitude -16.817` and `longitude -52.079` from January 1st, 2017 to December 31st, 2019, using the `ts` method:

In [ ]:
time_series = cbers4_coverage.ts(attributes=(red_band, nir_band),
                                 latitude=-16.817, longitude=-52.079,
                                 start_date="2017-01-01", end_date="2019-12-31")

# Plot the Time Series
<hr style="border:1px solid #0077b9;">

In [ ]:
time_series.plot()

# Scatter Plot
<hr style="border:1px solid #0077b9;">

Let's see the time series values:

In [ ]:
print(time_series.values(red_band))
print()
print(time_series.values(nir_band))

Performing a scatterplot between `red` and `NIR` Time Series we can see the correlation of theese bands through the time for the selected pixel:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(time_series.values(red_band), time_series.values(nir_band), alpha=0.5)
plt.title('Scatter plot')
plt.xlabel('Red')
plt.ylabel('NIR')
plt.show()

Time series of spatially-close points are likely to be similar. We can verify this by comparing our first time series to a time series extracted from `latitude -16.819` and `longitude -52.079` (also ranging from January 1st, 2017 to December 31st, 2019):

In [ ]:
time_series2 = cbers4_coverage.ts(attributes=(red_band, nir_band),
                                  latitude=-16.819, longitude=-52.079,
                                  start_date="2017-01-01", end_date="2019-12-31")

In [ ]:
plt.scatter(time_series.values(nir_band), time_series2.values(nir_band), alpha=0.5)
ident = [0.0, max(time_series.values(nir_band))] # Reference Line
plt.plot(ident,ident, color='red', ls='--')
plt.title('Scatter plot')
plt.xlabel('NIR TS1')
plt.ylabel('NIR TS2')
plt.show()

If all points were positioned on the red line, the time series would be equal. However, since they are similar, the points are located close to the red line, meaning they present close values.

# Calculate Median Time Series
<hr style="border:1px solid #0077b9;">

Another application would be: given a set of Time Series we may want to extract the median time series, which normally is the one with less noise.

Let's start by acquiring a few time series from an agricultural region, in this case a fixed `longitude -53.989` but a variable `latitude` ranging from `-16.905` until `-16.955` by a rate of `-0.01`, considering images from January 1st, 2017 to December 31st, 2019.

In [ ]:
import numpy

In [ ]:
agriculture_time_series = []
for latitude in numpy.arange(-16.905,-16.955,-0.01):
    time_series = cbers4_coverage.ts(attributes=(nir_band),
                                     latitude=float(latitude), longitude=-53.989,
                                     start_date="2017-01-01", end_date="2019-12-31")
    agriculture_time_series.append(time_series.values(nir_band))

This loop provides a total of five time series:

In [ ]:
len(agriculture_time_series)

The `Numpy` library provides the `median()` method, which calculates the median value of an array, and since we want to obtain the median value among the different time series, we can use the parameter `axis=0`:

In [ ]:
median = numpy.median(agriculture_time_series, axis=0)

In [ ]:
median

Now let's plot the original time series, in `grey` and the median time series, in `blue`:

In [ ]:
for i in range(len(agriculture_time_series)):
    plt.plot(agriculture_time_series[i], color='grey', alpha=0.5)
plt.plot(median, color='blue', linewidth=2)
plt.show()

We can visually note that the `blue` time series is centered in comparison to the `grey` ones.

# Time Series Smoothing
<hr style="border:1px solid #0077b9;">

Smoothing Algorithms are helpful to reduce time series noise, one of the most used smoothing algorithm is Savitz Golay, which has an implementation on the `scipy` library:

In [ ]:
from scipy.signal import savgol_filter

In [ ]:
median_smoothed = savgol_filter(median, window_length = 9, polyorder = 2)

<div style="text-align: center;  margin-left: 25%; margin-right: 25%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>Note:</b> The <em>Savitz Golay</em> algorithm uses a window_length and a polynomial order as parameters. You can change these values to see the impacts on the smoothed time series.
</div>

Now let's see the difference between the original time series and the smoothed one:

In [ ]:
plt.plot(median, color='blue')
plt.plot(median_smoothed, color='red')
plt.show()

We can observe that the smoothed time series (red) has less spikes than the original one (blue).

# References
<hr style="border:1px solid #0077b9;">

- [Python Client Library for Web Time Series Service - User Guide](https://wtss.readthedocs.io/en/latest/index.html)


- [Python Client Library for Web Time Series Service - GitHub Repository](https://github.com/brazil-data-cube/wtss.py)


- [WTSS OpenAPI 3 Specification](https://github.com/brazil-data-cube/wtss-spec)


- VINHAS, L.; QUEIROZ, G. R.; FERREIRA, K. R.; CÂMARA, G. [Web Services for Big Earth Observation Data](http://www.seer.ufu.br/index.php/revistabrasileiracartografia/article/view/44004). Revista Brasileira de Cartografia, v. 69, n. 5, 18 maio 2017.

# See also the following Jupyter Notebooks
<hr style="border:1px solid #0077b9;">

* [Introduction to the Web Time Series Service (WTSS)](./wtss-introduction.ipynb)